# Julia and structural geology 
Started 2019-07-05

This is a notebook for starting investigating the Julia tools applicable for structural geology and field mapping processings.

To start with, we consider the sources listing packages of interest for structural geology and field mapping.
Available sites are:
- Julia Observer: https://juliaobserver.com/packages
- JuliaGeo: https://github.com/JuliaGeo

I first consider the GDAL package, available at https://github.com/JuliaGeo/GDAL.jl

It can be added with the following commands:

In [1]:
using Pkg
Pkg.add("GDAL")
Pkg.test("GDAL")

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`


┌ Warning: Some registries failed to update:
│     — /home/mauro/.julia/registries/General — failed to fetch from repo
└ @ Pkg.Types /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.1/Pkg/src/Types.jl:1269


 Resolving package versions...
  Updating `~/.julia/environments/v1.1/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.1/Manifest.toml`
 [no changes]
   Testing GDAL
    Status `/tmp/tmplKt8xq/Manifest.toml`
  [b99e7846] BinaryProvider v0.5.5
  [add2ef01] GDAL v0.2.0
  [2a0f44e3] Base64  [`@stdlib/Base64`]
  [8ba89e20] Distributed  [`@stdlib/Distributed`]
  [b77e0a4c] InteractiveUtils  [`@stdlib/InteractiveUtils`]
  [8f399da3] Libdl  [`@stdlib/Libdl`]
  [56ddb016] Logging  [`@stdlib/Logging`]
  [d6f4376e] Markdown  [`@stdlib/Markdown`]
  [9a3f8284] Random  [`@stdlib/Random`]
  [ea8e919c] SHA  [`@stdlib/SHA`]
  [9e88b42a] Serialization  [`@stdlib/Serialization`]
  [6462fe0b] Sockets  [`@stdlib/Sockets`]
  [8dfed614] Test  [`@stdlib/Test`]


┌ Info: GDAL 2.2.4, released 2018/03/19
│ 159 GDAL drivers found
└ 46 OGR drivers found


   Testing GDAL tests passed 


CPLDestroyMutex: Error = 16 (Dispositivo o risorsa occupata)


In [2]:
using GDAL

In [11]:
gdalinfo_path = "/usr/bin/gdalinfo"

"/usr/bin/gdalinfo"

In [15]:
src_geotif = "/home/mauro/Documenti/projects/jeo/example_data/mt_alpi/mtAlpi_aster.tif" 

"/home/mauro/Documenti/projects/jeo/example_data/mt_alpi/mtAlpi_aster.tif"

In [16]:
run(`$(gdalinfo_path) $(src_geotif)`)

Driver: GTiff/GeoTIFF
Files: /home/mauro/Documenti/projects/jeo/example_data/mt_alpi/mtAlpi_aster.tif
Size is 213, 260
Coordinate System is:
PROJCS["WGS 84 / UTM zone 33N",
    GEOGCS["WGS 84",
        DATUM["WGS_1984",
            SPHEROID["WGS 84",6378137,298.257223563,
                AUTHORITY["EPSG","7030"]],
            AUTHORITY["EPSG","6326"]],
        PRIMEM["Greenwich",0,
            AUTHORITY["EPSG","8901"]],
        UNIT["degree",0.0174532925199433,
            AUTHORITY["EPSG","9122"]],
        AUTHORITY["EPSG","4326"]],
    PROJECTION["Transverse_Mercator"],
    PARAMETER["latitude_of_origin",0],
    PARAMETER["central_meridian",15],
    PARAMETER["scale_factor",0.9996],
    PARAMETER["false_easting",500000],
    PARAMETER["false_northing",0],
    UNIT["metre",1,
        AUTHORITY["EPSG","9001"]],
    AXIS["Easting",EAST],
    AXIS["Northing",NORTH],
    AUTHORITY["EPSG","32633"]]
Origin = (580814.582700000028126,4444299.458399999886751)
Pixel Size = (27.275953521126741,-

Process(`/usr/bin/gdalinfo /home/mauro/Documenti/projects/jeo/example_data/mt_alpi/mtAlpi_aster.tif`, ProcessExited(0))